In [7]:
import os
from dotenv import load_dotenv
from groq import Groq

In [8]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
print(groq_api_key)

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


In [9]:
client = Groq(
    api_key=groq_api_key,
)

#note que o groq pega uma lista de mensagens, não uma única string
chat_completion = client.chat.completions.create(
    #primeiro parametro é a lista de mensagens
    messages=[

        {

            "role": "user",

            "content": "Explain the importance of fast language models",

        }

    ],
    #segundo parametro é o modelo de linguagem que será usado (é possível setar outro modelo que a API hospeda)
    model="llama-3.3-70b-versatile",
    
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in today's technology landscape, and their importance can be seen in several areas:

1. **Improved User Experience**: Fast language models enable applications to respond quickly to user queries, providing a seamless and efficient experience. This is particularly important in real-time applications such as chatbots, virtual assistants, and language translation software.
2. **Enhanced Productivity**: By processing and generating text quickly, fast language models can significantly boost productivity in various industries, including customer service, content creation, and data analysis.
3. **Real-Time Applications**: Fast language models are essential for real-time applications such as:
	* Sentiment analysis: quickly analyzing user feedback and sentiment.
	* Chatbots: responding to user queries in real-time.
	* Live translation: translating languages in real-time for international communication.
	* Summarization: generating summaries of long documents or a

In [10]:
#Agora vamos criar uma classe de Agente
class Agent:
    def __init__(self, client, system):
        self.client = client #o cliente do groq
        self.system = system #o system prompt
        self.messages = [] #lista de mensagens
        if self.messages is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content


In [11]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
Returns the mass of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of the Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with:

Observation: 5.972e24

Thought: Now I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

If you have the answer, output it as the Answer

Answer: The mass of Earth times 2 is 1.1944e25 kg

Now it's your turn!

""".strip()

In [12]:
#tools
def calculate(operation):
    return eval(operation)


def get_planet_mass(planet):
    match planet.lower():
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case _:
            return 0.0



In [13]:
#inciando o agente
neil_tyson = Agent(client, system_prompt)

In [15]:
result = neil_tyson("What is the mass of Earth times 5?")

print(result)

Thought: Now that I have the mass of Earth, I can calculate the mass of Earth times 5 by multiplying the mass of Earth by 5.

Action: calculate: 5.972e24 * 5
PAUSE


In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns the mass of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of the Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with:\n\nObservation: 5.972e24\n\nThought: Now I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nIf you have the answer, output it as the Answer\n\nAnswer: The mass of Earth tim

In [ ]:
calculate = neil_tyson("calculate: 5.972e24 * 5")